In [161]:
import openpyxl
import pprint
import re
import pandas as pd

#Excel読み込み
wb1 = openpyxl.load_workbook("moto_sample.xlsx")
wb2 = openpyxl.load_workbook("seikyu.xlsx")

#シートネーム全部リスト化
wb1sheet = [sheetname for sheetname in wb1.sheetnames]#請求元サンプル入ってる。
wb2sheet = [sheetname for sheetname in wb2.sheetnames]#請求書一覧データ

#3行目をカラムとしてリスト化
col = []
for row in sheet1.iter_cols():
    for cell in row:
        if cell.row == 3:
            col.append(cell.value)
print(col)

#各シートデータ、行最大値リスト化
sheet_list = []
max_row_list = []
for i,sheet in enumerate(wb2sheet):
    if re.search('請求一覧', sheet):
        sheet_row = wb2[wb2sheet[i]]
        sheet_list.append(sheet_row)
        max_row_list.append(sheet_row.max_row)

#列名置き換え用辞書型カラムデータ作成
dic_col = {}
for i,col in enumerate(col):
    dic_col[i] = col
 
print(dic_col)
print(sheet_list)
print(max_row_list)

#行情報取得関数（取得した行をリストに入れる）
def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                          max_col=end_col))

#顧客管理テーブルシートを読み込んでデータフレーム化
cs = wb2["顧客管理テーブル"]
cs_list = get_list_2d(cs,3,cs.max_row,1,2)

cs_df = pd.DataFrame(cs_list)
cs_df = cs_df.dropna(how='all')
#請求一覧と合体させたいので顧客ＣＤを相手先コードにする
cs_df.columns = ['相手先コード','顧客名']
display(cs_df)


['受注No.', '相手先名', '相手先コード', '商品名１', '商品名２', '検収日', '金額（税抜き）', '備考']
{0: '受注No.', 1: '相手先名', 2: '相手先コード', 3: '商品名１', 4: '商品名２', 5: '検収日', 6: '金額（税抜き）', 7: '備考'}
[<Worksheet "請求一覧2101">, <Worksheet "請求一覧2102">, <Worksheet "請求一覧2103">, <Worksheet "請求一覧2105">, <Worksheet "請求一覧2106">, <Worksheet "請求一覧2107">, <Worksheet "請求一覧2108">, <Worksheet "請求一覧2109">, <Worksheet "請求一覧2110">]
[13, 13, 13, 13, 13, 13, 13, 13, 13]


,相手先コード,顧客名
0,1001.0,アデックテクノロジー株式会社
1,1002.0,株式会社サンワテック
2,1003.0,プロト技研株式会社
3,1004.0,株式会社AICUBEシステムズ
4,1005.0,大川情報開発合同会社
5,2001.0,NGCネットワークス株式会社
6,2002.0,光栄情報サービス株式会社
7,3001.0,株式会社クスダ製作所
8,3002.0,大関西リサーチ株式会社
9,4001.0,学校法人 さくら学園


In [194]:
from datetime import datetime
import calendar

seikyu_all = pd.DataFrame()
for i,sheet in enumerate(sheet_list):
    l_2d = get_list_2d(sheet, 4, max1, 0, 8)
    name = sheet.title[:4]
    year = "20" + sheet.title[4:6]
    month = sheet.title[6:]
    _, lastday = calendar.monthrange(int(year),int(month))
    date = datetime(int(year),int(month),lastday)
    date_ym = date.strftime('%Y-%m-%d')
    df = pd.DataFrame(l_2d)
    seikyu_df = df.rename(columns=dic_col)
    seikyu_df = seikyu_df.dropna(how='all')
    seikyu_df['年月'] = date_ym
    #seikyu_df = seikyu_df.set_index('年月',drop = True)
    seikyu_all = pd.concat([seikyu_all, seikyu_df])

#display(seikyu_all)

seikyu_sitei = pd.DataFrame()

#seikyu_sitei = seikyu_all[(seikyu_all["相手先コード"] == 1005) &  (seikyu_all["年月"] == "2021-02")]
seikyu_sitei = seikyu_all[seikyu_all["相手先コード"] == 1005]

#請求一覧、顧客一覧データフレームをマージする。
merge_df = pd.merge(seikyu_all,cs_df)
del merge_df['']
#display(seikyu_sitei)
#display(merge_df)

#請求元シート読み込み
moto_sheet = wb1["請求書元"]
row_list = ['A','B','E','F','G','H']
output_df = pd.DataFrame()
def seikyu_output(data):
    data = data[(data["相手先コード"] == 1005) &  (data["年月"] == "2021-03-31")].sort_values(by="受注No.",ascending = False)
    data.del
    no = data['年月'].iloc[0]
    date = data["年月"].iloc[0]
    no = no[2:4] + no[5:7] + "01"
    name = data['顧客名'].iloc[0] + "　御中"
    print(no,date,name)
    for i,col in enumerate(data):
        if i < 8:
            exlatai = 18
            for j,atai in enumerate(data[col]):
                exlatai = exlatai + (j*2)
                cell = row_list[j] + str(exlatai)
                print(cell)
                '''
                for k,cell in enumerate(row_list):
                    cell = cell + str(exlatai)
                    moto_sheet[cell]
                    print(cell)
                '''
    return data

output_df = seikyu_output(merge_df)
display(output_df)
#display(output_df.iloc[0:,0:7])
#pprint.pprint(l_2d, width=40)

210301 2021-03-31 大川情報開発合同会社　御中
A18
B20
A18
B20
A18
B20
A18
B20
A18
B20
A18
B20
A18
B20
A18
B20


,受注No.,相手先名,相手先コード,商品名１,商品名２,検収日,金額（税抜き）,備考,年月,顧客名
3,210201.0,None,1005.0,顧問料,None,31.0,80000.0,2021.2月分,2021-03-31,大川情報開発合同会社
2,201203.0,None,1005.0,社内教育企画,None,30.0,150000.0,None,2021-03-31,大川情報開発合同会社
